In [1]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
from tqdm import tqdm

In [2]:
class DataPreprocessor:
    def __init__(self, filepath):
        self.filepath = filepath
        self.label_encoder = LabelEncoder()

    def load_data(self):
        self.data = pd.read_excel(self.filepath, header=0)
        self.features = self.data.iloc[:, :-1]
        self.labels = self.data.iloc[:, -1]

    def clean_data(self):
        self.features = self.features.apply(pd.to_numeric, errors='coerce')
        self.features = self.features.fillna(self.features.mean())

    def encode_labels(self):
        self.labels = pd.to_numeric(self.labels, errors='coerce')
        valid_indices = self.labels.dropna().index
        self.labels = self.labels.loc[valid_indices].astype(int)
        self.features = self.features.loc[valid_indices]
        self.labels = self.label_encoder.fit_transform(self.labels)

    def scale_features(self):
        scaler = StandardScaler()
        self.features = scaler.fit_transform(self.features)

    def split_data(self):
        return train_test_split(self.features, self.labels, test_size=0.2, random_state=42)

    def preprocess(self):
        self.load_data()
        self.clean_data()
        self.encode_labels()
        self.scale_features()
        return self.split_data()


In [3]:
from sklearn.exceptions import NotFittedError
from sklearn.metrics import accuracy_score

class MLPFaultModel:
    def __init__(self, num_inputs, learning_rate=0.001, alpha=0.001, hidden_layer_sizes=(100, 50), max_iter=2000):
        self.model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, max_iter=max_iter, alpha=alpha, random_state=1, learning_rate_init=learning_rate, warm_start=True)

    def train(self, X_train, y_train):
        self.model.fit(X_train, y_train)

    def evaluate(self, X_test, y_test):
        predictions = self.model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        report = classification_report(y_test, predictions)
        return accuracy, report

    def predict_proba(self, X):
        return self.model.predict_proba(X)

In [4]:
class MLPFaultModelManager:
    def __init__(self, input_dim, num_conditions, params):
        self.models = {condition: MLPFaultModel(input_dim, **params) for condition in range(num_conditions)}

    def train_models(self, X_train, y_train):
        for condition, model in self.models.items():
            binary_labels = (y_train == condition).astype(int)
            model.train(X_train, binary_labels)

    def evaluate_models(self, X_test, y_test):
        evaluations = {}
        for condition, model in self.models.items():
            binary_labels = (y_test == condition).astype(int)
            accuracy, report = model.evaluate(X_test, binary_labels)
            evaluations[condition] = {'accuracy': accuracy, 'report': report}
        return evaluations

In [5]:
class ResidualCalculator:
    def __init__(self, models):
        self.models = models

    def calculate_residuals(self, X, y):
        residuals = {}
        for condition, model in self.models.items():
            probabilities = model.predict_proba(X)[:, 1]
            actual = (y == condition).astype(int)
            residuals[condition] = np.abs(probabilities - actual)
        return residuals

In [6]:
class DynamicBayesianNetwork:
    def __init__(self, conditions):
        self.conditions = conditions
        self.model = BayesianNetwork()
        self.setup_model()
        self.inference = VariableElimination(self.model)

    def setup_model(self):
        condition_nodes = ['Condition_' + str(cond) for cond in self.conditions]
        self.model.add_nodes_from(['State'] + condition_nodes)
        self.model.add_edges_from([('State', node) for node in condition_nodes])
        cpd_state = TabularCPD(variable='State', variable_card=2, values=[[0.5], [0.5]])
        self.model.add_cpds(cpd_state)
        for node in condition_nodes:
            cpd = TabularCPD(variable=node, variable_card=2, values=[[0.95, 0.05], [0.05, 0.95]], evidence=['State'], evidence_card=[2])
            self.model.add_cpds(cpd)
        assert self.model.check_model(), "Model configuration errors"

    def integrate_with_dbn(self, evidence):
        return self.inference.query(variables=['State'], evidence=evidence)

In [7]:
class ModelEvaluator:
    def __init__(self, models):
        self.models = models

    def evaluate_models(self, X_test, y_test):
        evaluations = {}
        for condition, model in self.models.items():
            accuracy, report = model.evaluate(X_test, (y_test == condition).astype(int))
            evaluations[condition] = {'accuracy': accuracy, 'report': report}
        return evaluations

    def predict_random_samples(self, X_test, y_test, n_samples=500):
        indices = np.random.choice(len(X_test), size=n_samples, replace=False)
        predictions = [np.argmax([model.predict_proba(X_test[idx:idx+1])[:, 1] for model in self.models.values()], axis=0) for idx in indices]
        actuals = y_test[indices]
        accuracy = accuracy_score(actuals, predictions)
        results_df = pd.DataFrame({
            'Sample Index': indices,
            'Predicted Class': predictions,
            'Actual Class': actuals
        })
        return results_df, accuracy

In [8]:
def preprocess_data(data_preprocessor):
    # Load, clean, encode, and scale the data
    X_train, X_test, y_train, y_test = data_preprocessor.preprocess()
    # Further split X_train and y_train into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test


In [9]:
from joblib import Parallel, delayed
import tqdm
from tqdm import tqdm

def train_and_evaluate(params, X_train, y_train, X_test, y_test, num_conditions):
    learning_rate, alpha, hidden_layer_sizes, max_iter = params
    model_manager = MLPFaultModelManager(X_train.shape[1], num_conditions, {
        'learning_rate': learning_rate,
        'alpha': alpha,
        'hidden_layer_sizes': hidden_layer_sizes,
        'max_iter': max_iter
    })
    model_manager.train_models(X_train, y_train)
    evaluator = ModelEvaluator(model_manager.models)
    final_evaluations = evaluator.evaluate_models(X_test, y_test)
    accuracy = np.mean([eval['accuracy'] for eval in final_evaluations.values()])
    
    # Calculate residuals and integrate feedback from DBN
    residual_calculator = ResidualCalculator(model_manager.models)
    residuals = residual_calculator.calculate_residuals(X_test, y_test)
    feedback_evidence = {f'Condition_{i}': residuals[i].mean() > 0.5 for i in range(num_conditions)}
    dbn = DynamicBayesianNetwork(range(num_conditions))
    feedback_results = dbn.integrate_with_dbn(feedback_evidence)
    state_prob = feedback_results.values[1]  # Assuming state '1' indicates the need for adjustment
    
    return accuracy, params, state_prob

def main():
    filepath = r'C:\Users\SIDDHARTH SINGH\Downloads\Untitled Folder\Merged_dataset.xlsx'
    data_preprocessor = DataPreprocessor(filepath)
    X_train, X_test, y_train, y_test = data_preprocessor.preprocess()

    num_conditions = len(np.unique(y_train))
    best_accuracy = 0
    best_params = {}
    best_state_prob = 0

    param_grid = [(lr, a, hs, mi) for lr in [0.001, 0.01]
                                    for a in [0.001, 0.01]
                                    for hs in [(100, 50), (50, 25)]
                                    for mi in [500, 1000]]

    results = Parallel(n_jobs=-1, verbose=10)(
        delayed(train_and_evaluate)(params, X_train, y_train, X_test, y_test, num_conditions) for params in tqdm(param_grid, desc="Optimizing hyperparameters"))

    for result in results:
        accuracy, params, state_prob = result
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = params
            best_state_prob = state_prob
            print(f"New best parameters found: {params} with accuracy: {accuracy:.2%} and state probability: {state_prob:.2f}")

    print("Best parameters:", best_params)
    print("Best accuracy:", best_accuracy)
    print("Best state probability indicating need for adjustments:", best_state_prob)

if __name__ == "__main__":
    main()


Optimizing hyperparameters: 100%|██████████████████████████████████████████████████████| 16/16 [00:00<00:00, 88.40it/s]
[Parallel(n_jobs=-1)]: Done   3 out of  16 | elapsed:  1.4min remaining:  6.1min
[Parallel(n_jobs=-1)]: Done   5 out of  16 | elapsed:  2.1min remaining:  4.7min
[Parallel(n_jobs=-1)]: Done   7 out of  16 | elapsed:  2.2min remaining:  2.8min
[Parallel(n_jobs=-1)]: Done   9 out of  16 | elapsed:  2.2min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done  11 out of  16 | elapsed:  2.2min remaining:   60.0s
[Parallel(n_jobs=-1)]: Done  13 out of  16 | elapsed:  2.3min remaining:   31.5s


New best parameters found: (0.001, 0.001, (100, 50), 500) with accuracy: 99.54% and state probability: 0.00
Best parameters: (0.001, 0.001, (100, 50), 500)
Best accuracy: 0.9954325421611493
Best state probability indicating need for adjustments: 3.466908540044183e-21


[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:  2.3min finished
